In [1]:
!pip install requests

In [1]:
import numpy as np
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

In [ ]:
# goals
# - line history
# - % bets
# - % money
# NFL side and total

In [2]:
# scraping function
def scrape_for_soup(base_url, url):
    # send GET request
    response = requests.get(base_url+url)

    # status check
    if response.status_code != 200:
        return(f"Error: {response.status_code}")

    # parse HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [3]:
base_url = 'https://www.sportsbookreview.com'
url = '/betting-odds/nfl-football/'
soup = scrape_for_soup(base_url, url)

# parse links for game spread and spread history
games = soup.find_all('div', id='tbody-nfl')
main_links = set()
line_links = set()
for game in games:
    for a_tag in game.find_all('a', href=True):
        link = a_tag['href']
        if 'scores' in link:
            main_links.add(link)
        if 'line-history' in link:
            line_links.add(link)

In [4]:
main_links

{'/scores/nfl-football/matchup/304948/',
 '/scores/nfl-football/matchup/304951/',
 '/scores/nfl-football/matchup/304954/',
 '/scores/nfl-football/matchup/304962/',
 '/scores/nfl-football/matchup/304964/',
 '/scores/nfl-football/matchup/304966/',
 '/scores/nfl-football/matchup/304969/',
 '/scores/nfl-football/matchup/304973/',
 '/scores/nfl-football/matchup/304977/',
 '/scores/nfl-football/matchup/304982/',
 '/scores/nfl-football/matchup/304985/',
 '/scores/nfl-football/matchup/304993/',
 '/scores/nfl-football/matchup/304995/',
 '/scores/nfl-football/matchup/304997/',
 '/scores/nfl-football/matchup/304998/',
 '/scores/nfl-football/matchup/305011/'}

In [6]:
print(base_url+game_url)

https://www.sportsbookreview.com/scores/nfl-football/matchup/304917/


In [5]:
# iterate through each link to scrape target data
game_url = list(main_links)[0]
# send request
game_soup = scrape_for_soup(base_url, game_url)

# spread bets
spread = game_soup.find('div', text=re.compile('Spread'))
home_perc = spread.next_sibling.get_text()[:-1]
away_perc = spread.previous_sibling.get_text()[:-1]

# over/under bets
total_bets = game_soup.find('div', text=re.compile('Total'))
over_perc = total_bets.previous_sibling.get_text()[:-3]
under_perc = total_bets.next_sibling.get_text()[:-3]
# pull total
tot_div = game_soup.find('div', class_='row GameMatchup_mainColumn__cILb0')
# find the span that contains the best total for the over and strip the first element (second element is odds if needed)
total = float(tot_div.find('h4', text='Over').find_parent('div').find_next_sibling('div').find('span').contents[0].strip())

# teams and spread
away_div = game_soup.find('div', class_=re.compile('TeamName'))
home_div = game_soup.find('div', class_=re.compile('TeamName')).findNext('div', class_=re.compile('TeamName'))
home_team = re.findall('.+?(?=\+|-|PK)',home_div.get_text())[0]
away_team = re.findall('.+?(?=\+|-|PK)',away_div.get_text())[0]
away_spread = away_div.find(text=re.compile('\+?\-?\d+\.?\d*'))
home_spread = home_div.find(text=re.compile('\+?\-?\d+\.?\d*'))
print(f'The spread is {home_spread}')
print(f'% bets {home_team} {home_spread}: {home_perc}')
print(f'% bets {away_team} {away_spread} % bets: {away_perc} \n')
print(f'The total is {str(total)}')
print(f'% bets over {str(total)}: {over_perc}')
print(f'% bets under {str(total)}:{under_perc}')

The spread is +3
% bets SeattleSeahawks +3: 39
% bets BuffaloBills -3 % bets: 61 

The total is 45.0
% bets over 45.0: 71
% bets under 45.0:29


In [41]:
cols =['home_team','away_team', 'home_spread', 'home_percent', 'away_percent', 'total', 'over_percent', 'under_percent']
df = pd.DataFrame(columns=cols)

# iterate through each link to scrape target data
for game_url in list(main_links):
    # send request
    game_soup = scrape_for_soup(base_url, game_url)
    # pull number bets on spread and o/u

    # over/under bets
    total_bets = game_soup.find('div', text=re.compile('Total'))
    over_perc = float(total_bets.previous_sibling.get_text()[:-3])
    under_perc = float(total_bets.next_sibling.get_text()[:-3])
    # pull total
    tot_div = game_soup.find('div', class_='row GameMatchup_mainColumn__cILb0')
    # find the span that contains the best total for the over and strip the first element (second element is odds if needed)
    total = float(tot_div.find('h4', text='Over').find_parent('div').find_next_sibling('div').find('span').contents[0].strip())

    # teams
    away_div = game_soup.find('div', class_=re.compile('TeamName'))
    home_div = game_soup.find('div', class_=re.compile('TeamName')).findNext('div', class_=re.compile('TeamName'))
    home_team = re.findall('.+?(?=\+|-|PK)',home_div.get_text())[0]
    away_team = re.findall('.+?(?=\+|-|PK)',away_div.get_text())[0]
    # spread bets
    spread = game_soup.find('div', text=re.compile('Spread'))
    home_perc = float(spread.next_sibling.get_text()[:-1])
    away_perc = float(spread.previous_sibling.get_text()[:-1])
    home_spread = home_div.find(class_='h5 mb-2 text-secondary opacity-75').text
    # home_spread = home_div.find(text=re.compile('\+?\-?\d+\.?\d*'))
    away_spread = away_div.find(class_='h5 mb-2 text-secondary opacity-75').text
    # away_spread = away_div.find(text=re.compile('\+?\-?\d+\.?\d*'))
    
    # add data to df
    row = dict()
    row['home_team'] = home_team
    row['away_team'] = away_team
    row['home_spread'] = home_spread
    row['home_percent'] = home_perc
    row['away_percent'] = away_perc
    row['total'] = total
    row['over_percent'] = over_perc
    row['under_percent'] = under_perc

    df = df.append(row, ignore_index=True)

In [43]:
df

,home_team,away_team,home_spread,home_percent,away_percent,total,over_percent,under_percent
0,SeattleSeahawks,BuffaloBills,+3,39.0,61.0,45.0,71.0,29.0
1,L.A. RamsRams,MinnesotaVikings,+3,22.0,78.0,46.5,51.0,49.0
2,San Francisco49ers,DallasCowboys,-5.5,52.0,48.0,47.5,58.0,42.0
3,HoustonTexans,IndianapolisColts,-4.5,62.0,38.0,44.5,55.0,45.0
4,Las VegasRaiders,Kansas CityChiefs,+8.5,40.0,60.0,42.0,63.0,37.0
5,L.A. ChargersChargers,New OrleansSaints,-7,57.0,43.0,41.0,43.0,57.0
6,MiamiDolphins,ArizonaCardinals,-4.5,48.0,52.0,46.5,49.0,51.0
7,DetroitLions,TennesseeTitans,-11.5,70.0,30.0,44.5,69.0,31.0
8,CincinnatiBengals,PhiladelphiaEagles,-2.5,54.0,46.0,47.5,58.0,42.0
9,DenverBroncos,CarolinaPanthers,-12.5,67.0,33.0,40.0,39.0,61.0


In [9]:
# next: pull total history/movement if possible
line_url = list(line_links)[0]
line_soup = scrape_for_soup(base_url, line_url)

In [6]:
# TODO: pull results data to compare
# espn api
season_type = '2'
week_num = '2'
year = '2024'
url = f'https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard?dates={year}&seasontype={season_type}&week={week_num}'
result = requests.get(url)
json_result = result.json()

In [9]:
json_result

{'leagues': [{'id': '28',
   'uid': 's:20~l:28',
   'name': 'National Football League',
   'abbreviation': 'NFL',
   'slug': 'nfl',
   'season': {'year': 2024,
    'startDate': '2024-08-01T07:00Z',
    'endDate': '2025-02-15T07:59Z',
    'displayName': '2024',
    'type': {'id': '2',
     'type': 2,
     'name': 'Regular Season',
     'abbreviation': 'reg'}},
   'logos': [{'href': 'https://a.espncdn.com/i/teamlogos/leagues/500/nfl.png',
     'width': 500,
     'height': 500,
     'alt': '',
     'rel': ['full', 'default'],
     'lastUpdated': '2018-06-05T12:07Z'}],
   'calendarType': 'list',
   'calendarIsWhitelist': True,
   'calendarStartDate': '2024-08-01T07:00Z',
   'calendarEndDate': '2025-02-15T07:59Z',
   'calendar': [{'label': 'Preseason',
     'value': '1',
     'startDate': '2024-08-01T07:00Z',
     'endDate': '2024-09-05T06:59Z',
     'entries': [{'label': 'Hall of Fame Weekend',
       'alternateLabel': 'HOF',
       'detail': 'Aug 1-7',
       'value': '1',
       'startDa

In [15]:
div = game_soup.find('div', class_='row GameMatchup_mainColumn__cILb0')
print(div)

<div class="row GameMatchup_mainColumn__cILb0"><aside class="col-12 col-lg-3"><div class="mt-2"><h2 class="d-flex align-items-center justify-content-between mb-0 py-1 px-3 bg-dark h4" data-bs-theme="dark">Betting Consensus</h2><div class="d-flex justify-content-center pt-2"><div class="col-4 d-flex flex-column justify-content-center text-center text-primary fx-bold pb-3 GameMatchup_TeamName__hOY6B"><p class="h4">Dallas<br/>Cowboys</p><span class="h5 mb-2 text-secondary opacity-75">+5.5</span></div><div class="d-flex justify-content-center align-items-center mx-2 mt-4 rounded-circle fs-7 text-white bg-dark text-center GameMatchup_TagCircle__FABto">vs</div><div class="col-4 d-flex flex-column justify-content-center text-center text-primary fx-bold pb-3 GameMatchup_TeamName__hOY6B"><p class="h4">San Francisco<br/>49ers</p><span class="h5 mb-2 text-secondary opacity-75">-5.5</span></div></div><div class="d-flex flex-column w-100 border-top undefined Rows_Rows__wF9TM"><div class="d-flex ali